In [1]:
from pathlib import Path
import re

def parse_t_hat_ear(file_path):
    """
    Legge il file e crea un dizionario: {(flight, destination): time}
    """
    t_hat_dict = {}
    try:
        with open(file_path, "r") as f:
            in_t_hat = False
            for line in f:
                line = line.strip()
                if "param t_hat_ear" in line:
                    in_t_hat = True
                    continue
                if in_t_hat:
                    if ";" in line:
                        break
                    parts = line.split()
                    if len(parts) >= 3:
                        try:
                            flight = parts[0]
                            destination = parts[1]
                            time = float(parts[2])
                            t_hat_dict[(flight, destination)] = time
                        except:
                            pass
    except:
        pass
    return t_hat_dict

def parse_edge_distances(file_path):
    """
    Legge le distanze degli archi dal file: param: E: d:=
    Restituisce {(nodo1, nodo2): distanza}
    """
    edges = {}
    try:
        with open(file_path, "r") as f:
            in_edges = False
            for line in f:
                line = line.strip()
                if "param: E: d:=" in line:
                    in_edges = True
                    continue
                if in_edges:
                    if ";" in line:
                        break
                    parts = line.split()
                    if len(parts) >= 3:
                        try:
                            node1 = parts[0]
                            node2 = parts[1]
                            distance = float(parts[2])
                            edges[(node1, node2)] = distance
                        except:
                            pass
    except:
        pass
    return edges

def parse_velocities(file_path):
    """
    Legge v_min e v_max dal file e calcola v_avg
    """
    v_min = None
    v_max = None
    try:
        with open(file_path, "r") as f:
            for line in f:
                line = line.strip()
                if "param v_min" in line:
                    parts = line.split(":=")
                    if len(parts) == 2:
                        v_min = float(parts[1].strip())
                elif "param v_max" in line:
                    parts = line.split(":=")
                    if len(parts) == 2:
                        v_max = float(parts[1].strip())
                if v_min is not None and v_max is not None:
                    break
    except:
        pass
    
    if v_min is not None and v_max is not None:
        v_avg = (v_min + v_max) / 2.0
        return v_avg
    return 2.5  # Valore di default se non trovato

def extract_full_path(path_line):
    """
    Estrae il volo e l'intero percorso dal path
    Formato: 'volo nodo1 nodo2 nodo3' -> [volo, [nodo1, nodo2, nodo3]]
    """
    parts = path_line.strip().split()
    if len(parts) >= 2:
        flight = parts[0]
        nodes = parts[1:]
        return flight, nodes
    return None, None

def calculate_new_t_hat_ear(flight, new_nodes, t_hat_original, edges, v_avg):
    """
    Calcola i nuovi t_hat_ear per un path modificato.
    Formula: t_hat_ear[i,x] = t_hat_ear[i,prec(x)] + d[prec(x),x] / v_avg
    """
    new_t_hat = {}
    
    for i, dest in enumerate(new_nodes):
        # Cerca se esiste già un t_hat_ear per questa destinazione
        if (flight, dest) in t_hat_original:
            new_t_hat[dest] = t_hat_original[(flight, dest)]
        else:
            # Calcola il tempo usando la formula
            if i == 0:
                # Primo nodo: usa un tempo base o cerca il minimo esistente
                existing_times = [t for (f, d), t in t_hat_original.items() if f == flight]
                new_t_hat[dest] = min(existing_times) if existing_times else 0.0
            else:
                prev_node = new_nodes[i-1]
                # Tempo del nodo precedente
                prev_time = new_t_hat.get(prev_node, 0.0)
                
                # Cerca la distanza dell'arco
                if (prev_node, dest) in edges:
                    distance = edges[(prev_node, dest)]
                else:
                    # Se non trovato, prova l'arco inverso (grafo non orientato?)
                    distance = edges.get((dest, prev_node), 2.5)
                
                # Calcola: t_hat_ear[i,dest] = t_hat_ear[i,prev] + d[prev,dest] / v_avg
                new_t_hat[dest] = prev_time + (distance / v_avg)
    
    return new_t_hat

origem = Path("data/mercedesTD")
destinazione = Path("data/obtainedByHeur")

for file_origine in origem.rglob('*'):
    if file_origine.is_file():
        # Calcola il path relativo rispetto alla cartella origine
        relativo = file_origine.relative_to(origem)
        if "nDr1" in str(relativo):
            continue
        
        # Costruisce il path di destinazione mantenendo la struttura
        file_destinazione = destinazione / relativo

        # Crea le cartelle mancanti se necessario
        file_destinazione.parent.mkdir(parents=True, exist_ok=True)

        dir = file_origine.parent.name
        base_name = file_origine.stem
        if dir == "mercedesTD":
            pathFile = f"Path_{base_name}.txt"
        else:
            pathFile = f"{dir}/Path_{base_name}.txt"
        file_path = destinazione / pathFile
        contenuto = ""
        
        if file_path.exists():
            # Leggi i dati t_hat_ear dal file originale
            t_hat_original = parse_t_hat_ear(file_origine)
            
            # Leggi le distanze degli archi
            edges = parse_edge_distances(file_origine)
            
            # Leggi la velocità media
            v_avg = parse_velocities(file_origine)
            
            # Leggi i nuovi path completi
            new_paths_full = {}  # {flight: [list of nodes]}
            with open(file_path, "r") as f:
                for line in f:
                    flight, nodes = extract_full_path(line)
                    if flight and nodes:
                        if flight not in new_paths_full:
                            new_paths_full[flight] = []
                        new_paths_full[flight].extend(nodes)
            
            flights = set()
            flights_new_t_hat = {}  # {flight: {destination: time}}
            
            with open(file_origine,"r") as file:
                riga = file.readline()
                while riga:
                    if "fixedFlights" in riga:
                        # Leggo contentuto file originale e creo insieme di voli fissati
                        while ";" not in riga and riga:
                            riga = file.readline()
                            if ";" in riga:
                                break
                            flight_name = riga.strip().split(" ")[0]
                            flights.add(flight_name) 
                        
                        # Scrivo i voli fissati con path cambiato dall'euristica                         
                        contenuto +="set fixedFlights :=\n"
                        with open(file_path, "r") as file2:
                            riga2 = file2.readline()
                            while riga2:
                                flight = riga2.split(" ")[0]
                                if flight in flights:
                                    contenuto+= riga2.strip() + "\n"
                                    # Calcola i nuovi t_hat_ear per questo volo
                                    flight_path, nodes = extract_full_path(riga2)
                                    if flight_path and nodes:
                                        if flight not in flights_new_t_hat:
                                            flights_new_t_hat[flight] = {}
                                        # Calcola t_hat_ear per ogni nodo nel path
                                        calculated = calculate_new_t_hat_ear(flight, nodes, t_hat_original, edges, v_avg)
                                        flights_new_t_hat[flight].update(calculated)
                                riga2 = file2.readline()
                            contenuto +=";\n"
                    
                    # Cerca linee con t_hat_ear e aggiorna
                    elif "param t_hat_ear" in riga:
                        contenuto += riga.strip() + "\n"
                        riga = file.readline()
                        
                        # Ricostruisci il blocco t_hat_ear mantenendo tutto tranne i voli aggiornati,
                        # e sostituendo per i voli aggiornati tutti i (flight, destination)
                        blocco_t_hat = []
                        while riga:
                            riga_strip = riga.strip()
                            if ";" in riga_strip:
                                break
                            parts = riga_strip.split()
                            if len(parts) >= 3:
                                flight = parts[0]
                                # Mantieni le righe dei voli NON aggiornati
                                if flight not in flights_new_t_hat:
                                    blocco_t_hat.append(riga_strip)
                            else:
                                blocco_t_hat.append(riga_strip)
                            riga = file.readline()

                        # Aggiungi le nuove righe per i voli aggiornati
                        for flight, dest_times in flights_new_t_hat.items():
                            for dest, time_val in dest_times.items():
                                blocco_t_hat.append(f"{flight} {dest} {time_val}")

                        # Scrivi il blocco ricostruito
                        for linea in blocco_t_hat:
                            contenuto += linea + "\n"
                        contenuto += riga.strip() + "\n"  # la riga con ";"
                        riga = file.readline()
                        continue
                    
                    contenuto += riga.strip() + "\n"
                    riga = file.readline()
        else:
            print("file ",file_path," with updated path from the heuristic not found")
            continue       
        
        file_destinazione.write_text(contenuto)


file  data/obtainedByHeur/Path_metroplex41nDr0nDe1.txt  with updated path from the heuristic not found
file  data/obtainedByHeur/Path_metroplex88nDr0nDe3.txt  with updated path from the heuristic not found
file  data/obtainedByHeur/Path_metroplex2nDr0nDe1.txt  with updated path from the heuristic not found
file  data/obtainedByHeur/Path_metroplex75nDr0nDe3.txt  with updated path from the heuristic not found
file  data/obtainedByHeur/Path_metroplex2nDr0nDe1.txt  with updated path from the heuristic not found
file  data/obtainedByHeur/Path_metroplex75nDr0nDe3.txt  with updated path from the heuristic not found
file  data/obtainedByHeur/Path_metroplex67nDr0nDe1.txt  with updated path from the heuristic not found
file  data/obtainedByHeur/Path_metroplex79nDr0nDe3.txt  with updated path from the heuristic not found
file  data/obtainedByHeur/Path_metroplex67nDr0nDe1.txt  with updated path from the heuristic not found
file  data/obtainedByHeur/Path_metroplex79nDr0nDe3.txt  with updated path f

param nn:= 144;
set F:=
0
1
2
3
4
5
6
7
8
9
10
11
14
15
16
18
19
21
22
25
26
27
30
32
35
37
38
41
42
48
56
57
58
60
79
88
89
90
92
93
;
set NC:=
93
;
param: E: d:=
1 19 2.500000
21 0 2.500000
19 21 2.500000
21 4 2.500000
5 2 2.500000
6 9 2.500000
7 8 2.500000
29 10 2.500000
11 29 2.500000
30 11 2.500000
8 12 2.500000
12 28 2.500000
15 5 2.500000
16 23 2.500000
19 17 2.500000
17 20 2.500000
18 21 2.500000
14 26 2.500000
3 13 2.500000
131 6 1.500000
83 132 1.500000
133 84 1.500000
13 27 2.500000
27 33 2.500000
33 86 2.500000
86 88 2.500000
88 90 2.500000
90 92 2.500000
92 94 2.500000
94 96 2.500000
96 97 2.500000
97 101 2.500000
101 103 2.500000
103 105 2.500000
105 107 2.500000
107 109 2.500000
109 111 2.500000
111 113 2.500000
113 114 2.500000
114 115 2.500000
118 117 2.500000
117 116 2.500000
116 112 2.500000
112 110 2.500000
110 108 2.500000
108 106 2.500000
106 104 2.500000
104 102 2.500000
102 100 2.500000
100 95 2.500000
95 93 2.500000
93 91 2.500000
91 89 2.500000
87 85 2.500000
